In [30]:
import io, os, sys
from sys import stdin, stdout
 
# input = io.BytesIO(os.read(0,os.fstat(0).st_size)).readline
# def input(): return stdin.readline().strip()
# def read_int_list(): return list(map(int, input().split()))
# def read_int_tuple(): return tuple(map(int, input().split()))
# def read_int(): return int(input())
 
from itertools import permutations, chain, combinations, product
from math import factorial, gcd
from collections import Counter, defaultdict, deque
from heapq import heappush, heappop, heapify
from bisect import bisect_left
from functools import lru_cache            
import re


# 2

In [26]:
direcs = {
    ">": (0,1),
    "^": (-1,0),
    "v": (1,0),
    "<": (0,-1),
}

def draw(r_y, r_x, point_to_box, obstacles, num_y, num_x):
    for y in range(num_y):
        row = []
        for x in range(num_x):
            if (y,x) in point_to_box:
                row.append("O")
            elif (y,x) in obstacles:
                row.append("#")
            elif y==r_y and x==r_x:
                row.append("@")
            else:
                row.append(".")
        print("".join(row))
            


def execute_move(r_y, r_x, move, box_to_points, point_to_box, obstacles):
    n_y, n_x = r_y+direcs[move][0], r_x+direcs[move][1]
    if (n_y, n_x) in obstacles:
        return r_y, r_x
    elif (n_y, n_x) in point_to_box:
        if move_boxes(point_to_box[(n_y, n_x)], move, box_to_points, point_to_box, obstacles):
            return n_y, n_x
        else:
            return r_y, r_x
    else:
        return n_y, n_x


def move_boxes(box, move, box_to_points, point_to_box, obstacles):
    curr_boxes = set([box])
    to_move = set()
    # decide if movable
    # print("here")
    while curr_boxes:
        new_boxes = set()
        curr_points = [
            (point, box) for box in curr_boxes for point in box_to_points[box]
        ]
        # print(curr_points)
        for point, box in curr_points:
            to_move.add(box)
            new_point = (point[0]+direcs[move][0], point[1]+direcs[move][1])
            if new_point in obstacles:
                return False
            elif new_point in point_to_box and point_to_box[new_point] not in curr_boxes:
                new_boxes.add(point_to_box[new_point])
        curr_boxes = new_boxes
    # move
    temp = {}
    for box in to_move:
        points = box_to_points[box]
        new_points = []
        for point in points:
            new_point = (point[0]+direcs[move][0], point[1]+direcs[move][1])
            new_points.append(new_point)
            del point_to_box[point]
        box_to_points[box] = new_points
        for new_point in new_points:
            temp[new_point] = box
    for p, b in temp.items():
        point_to_box[p] = b
    return True



    
    

In [35]:
with open('inputs/sample', 'r') as f:
    lines = f.readlines()
    grid = []
    moves = ''
    for idx, line in enumerate(lines):
        if len(line) == 1:
            break
        grid.append(line[:-1])
    for line in lines[idx+1:]:
        moves += line[:-1]
    num_y = len(grid)
    num_x = len(grid[0])
    obstacles = set()
    box_to_points = {}
    point_to_box = {}
    r_y, r_x = -1,-1
    box = 0
    for y in range(num_y):
        for x in range(num_x):
            if grid[y][x] == '#':
                obstacles.add((y,2*x))
                obstacles.add((y,2*x+1))
            elif grid[y][x] == 'O':
                box_to_points[box] = [
                    (y,2*x), (y,2*x+1)
                ]
                point_to_box[(y,2*x)] = box
                point_to_box[(y,2*x+1)] = box
                box+=1
            elif grid[y][x] == '@':
                r_y, r_x = y, x*2
    for move in moves:
        r_y, r_x = execute_move(r_y, r_x, move, box_to_points, point_to_box, obstacles)
        # print(move, r_y, r_x)
        # draw(r_y, r_x, point_to_box, obstacles, num_y, num_x*2)

res=0
for box in box_to_points:
    p1 = box_to_points[box][0]
    p2 = box_to_points[box][1]
    if (p1[0]+p1[1])>(p2[0]+p2[1]):
        res += 100*p2[0] + p2[1]
    else:
        res += 100*p1[0] + p1[1]
print(res)

< 4 7
####################
##....OO....OO..OO##
##............OO..##
##..OOOO....OO..OO##
##...OO@......OO..##
##OO##....OO......##
##OO....OO....OO..##
##..OOOO..OO..OOOO##
##........OO......##
####################
v 5 7
####################
##....OO....OO..OO##
##............OO..##
##..OOOO....OO..OO##
##...OO.......OO..##
##OO##.@..OO......##
##OO....OO....OO..##
##..OOOO..OO..OOOO##
##........OO......##
####################
v 6 7
####################
##....OO....OO..OO##
##............OO..##
##..OOOO....OO..OO##
##...OO.......OO..##
##OO##....OO......##
##OO...@OO....OO..##
##..OOOO..OO..OOOO##
##........OO......##
####################
> 6 8
####################
##....OO....OO..OO##
##............OO..##
##..OOOO....OO..OO##
##...OO.......OO..##
##OO##....OO......##
##OO....@OO...OO..##
##..OOOO..OO..OOOO##
##........OO......##
####################
^ 5 8
####################
##....OO....OO..OO##
##............OO..##
##..OOOO....OO..OO##
##...OO.......OO..##
##OO##..@.OO......##
##OO